In [1]:
# import relevant libraries
import csv_diff
import pandas.io.json
import pandas as pd
import datetime
import sys
import csv

In [2]:
csv.field_size_limit(sys.maxsize)

131072

In [3]:
# load current and previous release data
data_previous_release = csv_diff.load_csv(open('data/2002_data_v2.csv'), key='ensembl_gene_id')
data_current_release = csv_diff.load_csv(open('data/20_04_data_v1.csv'), key='ensembl_gene_id') 

print("Records in the previous release data = %i" % len(data_previous_release))
print("Records in the current release data = %i" % len(data_current_release))

difference_in_release_data = len(data_current_release) - len(data_previous_release) 
print("Difference between the number of rows in the two release data files = %i" % (difference_in_release_data))

Records in the previous release data = 15157
Records in the current release data = 60564
Difference between the number of rows in the two release data files = 45407


In [4]:
# run CSV diff on data files (output is JSON as specified in the csv_diff docs)
tractability_data_csv_diff_results = csv_diff.compare(data_previous_release, data_current_release)

In [5]:
# convert JSON into DataFrame and save as CSV
normalised_csv_diff_data = pandas.io.json.json_normalize(tractability_data_csv_diff_results['changed'])
tractability_csv_diff_df = pd.DataFrame.from_dict(normalised_csv_diff_data, orient='columns')
print('Generated tractability diff file with %i records changed' % (len(tractability_csv_diff_df)))

Generated tractability diff file with 15157 records changed


In [6]:
# generate and save report report
today_date = datetime.datetime.now().strftime('%Y-%m-%d')
report_directory = 'reports/'
report_filename = today_date + '_' + 'tractability_data_csv_diff_report.csv'
tractability_csv_diff_df.to_csv(report_directory + report_filename, index=False)

In [7]:
tractability_csv_diff_df.columns

Index(['changes.Bcell_mean', 'changes.Bucket_2_ab', 'changes.Bucket_3_ab',
       'changes.Bucket_4_ab', 'changes.Bucket_4_sm', 'changes.Bucket_6_ab',
       'changes.Bucket_7_ab', 'changes.Bucket_8_PROTAC',
       'changes.Bucket_9_PROTAC', 'changes.Bucket_sum_PROTAC',
       'changes.Bucket_sum_ab', 'changes.Category_ab',
       'changes.Clinical_Precedence_ab', 'changes.Hepatocytes_mean',
       'changes.High_Quality_ChEMBL_compounds', 'changes.Max_halflife',
       'changes.MouseNeuorons_mean', 'changes.NKcell_mean',
       'changes.PDB_Known_Ligand', 'changes.PROTAC_location_Bucket',
       'changes.Signal_peptide', 'changes.Top_bucket_PROTAC',
       'changes.Top_bucket_ab', 'changes.Transmembrane',
       'changes.Uniprot_high_conf_loc', 'changes.Uniprot_med_conf_loc',
       'changes.full_id', 'changes.number_of_ubiquitination_sites',
       'changes.symbol', 'key'],
      dtype='object')

In [8]:
all_clinical_precedence_buckets = [
    'changes.Bucket_1_sm',
    'changes.Bucket_1_ab',
    'changes.Bucket_2_sm',
    'changes.Bucket_2_ab',
    'changes.Bucket_3_sm',
    'changes.Bucket_3_ab'
]

clinical_predence_buckets_in_csv_diff_file = ['key'] 

for bucket in all_clinical_precedence_buckets:
    if bucket in tractability_csv_diff_df.columns:
        clinical_predence_buckets_in_csv_diff_file.append(bucket)

print(clinical_predence_buckets_in_csv_diff_file)

['key', 'changes.Bucket_2_ab', 'changes.Bucket_3_ab']


In [10]:
tractability_clin_precedence_df = tractability_csv_diff_df.filter(clinical_predence_buckets_in_csv_diff_file)
tractability_clin_precedence_df.set_index('key', inplace=True, drop=True)
tractability_clin_precedence_df.head()

,changes.Bucket_2_ab,changes.Bucket_3_ab
key,,
ENSG00000128052,NaN,NaN
ENSG00000146648,NaN,NaN
ENSG00000141736,NaN,NaN
ENSG00000005844,NaN,NaN
ENSG00000140443,NaN,NaN


In [11]:
tractability_clin_precedence_df.dropna(how="all", inplace=True)
tractability_clin_precedence_df.head()

,changes.Bucket_2_ab,changes.Bucket_3_ab
key,,
ENSG00000182578,"[1, 0]","[0, 1]"
ENSG00000134352,"[1, 0]",NaN


In [12]:
print(len(tractability_clin_precedence_df))

2


In [13]:
today_date = datetime.datetime.now().strftime('%Y-%m-%d')
report_directory = 'reports/'
report_filename = today_date + '_' + 'tractability_data_clin_prec_only_csv_diff_report.csv'
tractability_clin_precedence_df.to_csv(report_directory + report_filename, index=True)